In [ ]:
# 1. Tải TF Object Detection APi vào thư mục models
# %cd /content/gdrive/MyDrive
# !git clone https://github.com/tensorflow/models.git

In [13]:
# %cd D:\Paper\Raspberrypi\Object_OD_OT\Predict_Mobile_tf

D:\Paper\Raspberrypi\Object_OD_OT\Predict_Mobile_tf


d:\Paper\Raspberrypi\Object_OD_OT\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
!pip uninstall Cython -y

In [2]:
# 2. Compile the protos -  
# Protocol Buffers (Protobuf) is a free and open-source cross-platform data format used to serialize structured data.
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

d:\Paper\Raspberrypi\Object_OD_OT\Predict_Mobile_tf\models\research


In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
# import re
# with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
#     s = f.read()

# with open('/content/models/research/setup.py', 'w') as f:
#     # Set fine_tune_checkpoint path
#     s = re.sub('tf-models-official>=2.5.1',
#                'tf-models-official==2.8.0', s)
#     f.write(s)

In [6]:
# Install the Object Detection API
# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install .

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

Processing d:\paper\raspberrypi\object_od_ot\predict_mobile_tf\models\research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached avro-python3-1.10.2.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached apache_beam-2.51.0-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached Cython-3.0.5-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached sacrebleu-2.2.0-py3-none-any.whl (116 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl.metadata (8.5 kB)
  Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached crcmod-1.7.tar.gz (89 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'don

ERROR: Could not find a version that satisfies the requirement tensorflow==2.8.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==2.8.0


In [8]:
!python D:\Paper\Raspberrypi\Object_OD_OT\Predict_Mobile_tf\models\research\object_detection\builders\model_builder_tf2_test.py

2023-11-16 10:33:33.735537: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Traceback (most recent call last):
  File "d:\Paper\Raspberrypi\Object_OD_OT\.venv\Lib\site-packages\object_detection\models\ssd_efficientnet_bifpn_feature_extractor.py", line 35, in <module>
    from official.legacy.image_classification.efficientnet import efficientnet_model
ModuleNotFoundError: No module named 'official.legacy'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Paper\Raspberrypi\Object_OD_OT\Predict_Mobile_tf\models\research\object_detection\builders\model_builder_tf2_test.py", line 24, in <module>
    from object_detection.builders import model_builder
  File "d:\Paper\Raspberrypi\Object_OD_OT\.venv\Lib\site-pack

In [7]:
import matplotlib
import matplotlib.pyplot as plt

import io
import os
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow.python.keras.layers.preprocessing'

**Build a detection model and load pre-trained model weights**
This sometimes takes a little while, please be patient!

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
!tar -xf centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz
!mv centernet_hg104_512x512_kpts_coco17_tpu-32/checkpoint models/research/object_detection/test_data/

In [ ]:
# Các hàm inference

import cv2
def run_inference_for_single_image(model, image):
  
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict